<h1> Predictor typos in most influential country </h1>


* <b>Reads each db containing n-typos (<i>df_wine_country_typos_n.csv</i>)</b>
* <b>Incrementally generates typos from 1 to 100%</b>
* <b>In each case, executes one hot encoding of categorical variables</b>
* <b>In each case, calculates HPO </b>
* <b>Retrieves base MSE of data with n-typos</b>
* ! <b>The number of experiments is limited by execution time constraint</b>

In [24]:
# imports
import copy
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split 
from sklearn import metrics
from sklearn import linear_model
import pandas as pd 
import numpy as np
from sklearn import metrics  
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint as sp_randint
from sklearn.preprocessing import LabelEncoder
import random
import string

%matplotlib inline

In [25]:
df_wine = pd.read_csv("df_wine_country_typos_2.csv", encoding = 'utf8', index_col=0)
df_wine = df_wine.reset_index()
#frequencies = df_wine['country'].value_counts()
#frequencies.sort_values
#print(frequencies)


In [26]:
df_wine_typos = pd.get_dummies(df_wine, columns=['country'])
df_wine_typos = pd.get_dummies(df_wine_typos, columns=['province'])
df_wine_typos = pd.get_dummies(df_wine_typos, columns=['region'])
df_wine_typos = pd.get_dummies(df_wine_typos, columns=['variety'])
df_wine_typos.head(4)

,level_0,index,price,points,year_of_wine,country_AASB,country_AGES,country_AIoS,country_AQSN,country_ASC,...,variety_Vitovska,variety_Viura,variety_Viura-Chardonnay,variety_Viura-Verdejo,variety_White Blend,variety_White Riesling,variety_Xarel-lo,variety_Zibibbo,variety_Zinfandel,variety_Zweigelt
0,0,0,14.0,87,2013,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,13.0,87,2013,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,2,65.0,87,2012,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,3,15.0,87,2011,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
data_sets = list()
n=50
for i in range(1, n+1):
    # Load the Typo Wine Dataset
    df_wine = pd.read_csv("df_wine_country_typos_{}.csv".format(i), encoding = 'utf8', index_col=0)
    df_wine = df_wine.reset_index()
    df_wine.head(4)
    
    # one hot encoding
    df_wine_typos = pd.get_dummies(df_wine, columns=['country'])
    df_wine_typos = pd.get_dummies(df_wine_typos, columns=['province'])
    df_wine_typos = pd.get_dummies(df_wine_typos, columns=['region'])
    df_wine_typos = pd.get_dummies(df_wine_typos, columns=['variety'])
    
    data_sets.append(df_wine_typos)

In [28]:
print(len(data_sets))

50


In [ ]:
# check columns number
print("[TEST]")
i = 1 
for df in data_sets:
    aux = copy.deepcopy(df)
    print("[OK] Data set contains {}% of typos in {} columns.".format(i , len(aux.columns)))
    i += 1
    

In [30]:
def get_hyperparameters(X_train, y_train):
    regressor3 = DecisionTreeRegressor()
    param_dist = {'max_depth': sp_randint(2,16),
                  'min_samples_split': sp_randint(2,16)}

    n_iter_search = 20
    clfrs = RandomizedSearchCV(regressor3, 
                               param_distributions=param_dist,
                               scoring='neg_mean_squared_error',
                               cv=5 , n_jobs=1, verbose=1,
                               n_iter=n_iter_search)
    clfrs.fit(X_train, y_train)
    params = clfrs.best_params_
    return params["max_depth"], params["min_samples_split"]

In [31]:
data_sets[0].head(4)

,level_0,index,price,points,year_of_wine,country_Argentina,country_Australia,country_BBS,country_BBpS,country_BSW,...,variety_Vitovska,variety_Viura,variety_Viura-Chardonnay,variety_Viura-Verdejo,variety_White Blend,variety_White Riesling,variety_Xarel-lo,variety_Zibibbo,variety_Zinfandel,variety_Zweigelt
0,0,0,14.0,87,2013,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,13.0,87,2013,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,2,65.0,87,2012,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,3,15.0,87,2011,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# RESULTS

In [ ]:
import time

results = list()
i = 1

for df in data_sets:
    aux = copy.deepcopy(df)
    print("[OK] Experiment {} contains {}% of typos in {} columns.".format(i, i, len(aux.columns)))
    start_time = time.time()
    # split data
    df_X = aux.drop('points', axis=1)
    df_Y = aux[['points']]
    X_train, X_test, y_train, y_test = train_test_split(df_X, df_Y, test_size=0.3, random_state=0)
    print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
    # get hyperparameters
    # max_depth, min_samples_split = get_hyperparameters(X_train, y_train)
    # print("max_depth: {}, min_samples_split: {}".format(max_depth, min_samples_split))
    
    # apply regression
    regressor = DecisionTreeRegressor()  
    regressor.fit(X_train, y_train) # Train the model using the training sets
    y_pred = regressor.predict(X_test) # Make predictions using the testing set

    # The evaluation metrics
    mae = metrics.mean_absolute_error(y_test, y_pred)
    mse = metrics.mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
    print('Mean Absolute Error:', mae)  
    print('Mean Squared Error:', mse)  
    print('Root Mean Squared Error:', rmse) 
    print("*****"*20)
    elapsed_time = time.time() - start_time
    
    # store results
    result = dict(
        n=i,
        columns=len(aux.columns),
        elapsed_time=int(elapsed_time),
        results=dict(mae=mae, mse=mse, rmse=rmse)
    )
    results.append(result)
    i += 1
#     if i == 3:
#         break

[OK] Experiment 1 contains 1% of typos in 2074 columns.
(68392, 2073) (29311, 2073) (68392, 1) (29311, 1)
Mean Absolute Error: 2.1787724744976287
Mean Squared Error: 8.782163692811572
Root Mean Squared Error: 2.963471560992542
****************************************************************************************************
[OK] Experiment 2 contains 2% of typos in 2513 columns.
(68392, 2512) (29311, 2512) (68392, 1) (29311, 1)
Mean Absolute Error: 2.1791477602265363
Mean Squared Error: 8.78438129030057
Root Mean Squared Error: 2.963845692727705
****************************************************************************************************
[OK] Experiment 3 contains 3% of typos in 2979 columns.
(68392, 2978) (29311, 2978) (68392, 1) (29311, 1)
Mean Absolute Error: 2.175701954897479
Mean Squared Error: 8.729964859609021
Root Mean Squared Error: 2.9546513939226435
****************************************************************************************************
[OK] Experiment 4

[OK] Experiment 26 contains 26% of typos in 10973 columns.
(68392, 10972) (29311, 10972) (68392, 1) (29311, 1)
Mean Absolute Error: 2.1636245778035548
Mean Squared Error: 8.577394152366006
Root Mean Squared Error: 2.928718858539687
****************************************************************************************************
[OK] Experiment 27 contains 27% of typos in 11255 columns.
(68392, 11254) (29311, 11254) (68392, 1) (29311, 1)


In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(results)